# Project Check-In Week 4: Logistic Regression

The below code is a simple demonstration of a logistic regression on the spotify dataset. To make sure the data we are working with has the proper format, we will use the data output by the dataclean.py script we generated in an earlier week. For this check in, we will attempt to classify tracks into one of two genres (for simplicity) based on other features in the data set.

In [1]:
# Import relevant libraries
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
# Load the data and check for duplicates
logistic_data_orig = pd.read_excel('../Data/clean_data.xlsx')

In [3]:
assert logistic_data_orig.duplicated("track_id").sum() == 0, "There are duplicates track_ids in the data"
logistic_data_orig["track_genre"].value_counts()

track_genre
chicago-house    998
cantopop         998
alt-rock         997
breakbeat        996
forro            996
                ... 
metal            230
punk             226
house            210
indie            132
reggaeton         74
Name: count, Length: 113, dtype: int64

Since we will be classifying the data into genres, it is important that each song only has one genre. The above line is used to ensure our data cleaning returned data such that each track only has one genre. The other part of the previous code block is used to see how many of each genre is in the data. This is important as the proportion of each gene will affect the accuracy of the model. If one genre is much more common than the other, the model may be biased towards that genre. For the sake of this check in, we will thus select two genres that are relatively close in number of tracks.

In [5]:
# Select a subset of the data s.t. we only have 2 classes
# We will use the 'genre' column to create the classes
genre_1 = "chicago-house" # will convert to be the positive class (1)
genre_2 = "cantopop" # will convert to be the negative class (0)

d = {
    genre_1: 1,
    genre_2: 0
}

# Split data into training and validation sets (no testing set for project check in)
# Ensure the proportion of each class is the same in the training and validation sets
# logistic_data = logistic_data_orig[logistic_data_orig["track_genre"].isin([genre_1, genre_2])]
# logistic_data = logistic_data.replace({"track_genre": {genre_1: 1, genre_2: 0}}).select_dtypes(include=[np.number]).drop(columns="Unnamed: 0")
# logistic_data_train = logistic_data.sample(frac=0.8, random_state=0)
# logistic_data_val = logistic_data.drop(logistic_data_train.index)

logistic_data = logistic_data_orig[logistic_data_orig["track_genre"].isin([genre_1, genre_2])]
logistic_data = logistic_data.replace({"track_genre": d}).select_dtypes(include=[np.number]).drop(columns="Unnamed: 0")

# With selected predictor variables:
logistic_data = logistic_data[['danceability', 'acousticness', 'instrumentalness','time_signature', 'track_genre']]

# Split data into training and validation sets - stratified split to ensure same proportion of classes in both sets
logistic_data_train, logistic_data_val = train_test_split(logistic_data, test_size=0.2, random_state=42, stratify=logistic_data["track_genre"])

# Check the proportion of classes in the training and validation sets
prop_zero_train = (logistic_data_train["track_genre"] == 0).sum() / len(logistic_data_train)
prop_zero_val = (logistic_data_val["track_genre"] == 0).sum() / len(logistic_data_val)
assert np.isclose(prop_zero_train, prop_zero_val, atol=0.05), "Proportion of classes in training and validation sets are not the same"

C:\Users\hbken\AppData\Local\Temp\ipykernel_23272\1682029758.py:19: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  logistic_data = logistic_data.replace({"track_genre": d}).select_dtypes(include=[np.number]).drop(columns="Unnamed: 0")


In [6]:
logistic_data_val

,danceability,acousticness,instrumentalness,time_signature,track_genre
12835,0.698,0.156000,0.892000,4,1
12460,0.643,0.011500,0.000615,4,1
12498,0.813,0.008290,0.010100,4,1
11283,0.569,0.258000,0.000000,4,0
12248,0.783,0.000541,0.967000,4,1
...,...,...,...,...,...
11431,0.692,0.846000,0.000000,4,0
11857,0.600,0.086500,0.000000,4,0
12685,0.669,0.181000,0.120000,4,1
11438,0.569,0.001020,0.000029,4,0


In [5]:
# Compute logistic regression over entire training data set
lr_all = LogisticRegression(solver='liblinear')
lr_all.fit(X=logistic_data_train.drop(columns="track_genre"), y=logistic_data_train["track_genre"])
lr_all.intercept_, lr_all.coef_

(array([-0.00075326]),
 array([[-1.17306856e-01,  1.30401212e-05,  1.14492578e-03,
          1.83210242e-03,  2.81094278e-03,  1.51998666e-02,
         -2.53195012e-03,  2.77573203e-04, -4.13183266e-03,
          3.21564546e-03, -3.89094249e-04,  1.51888233e-03,
         -6.54353370e-03, -2.64092033e-03]]))

In [6]:
# Test the model on a subset of all observations in the validation set
X_val = logistic_data_val.drop(columns="track_genre")
y_val = logistic_data_val["track_genre"]

pred_val = pd.DataFrame({"actual": y_val, "predicted": lr_all.predict(X_val), "prob": lr_all.predict_proba(X_val)[:,1]})
# pred_val.replace({1: genre_1, 0: genre_2}, inplace=True)
pred_val

,actual,predicted,prob
12835,1,1,0.935832
12460,1,1,0.544193
12498,1,1,0.933830
11283,0,0,0.051029
12248,1,1,0.771968
...,...,...,...
11431,0,0,0.022011
11857,0,0,0.021958
12685,1,1,0.997073
11438,0,1,0.607564


In [7]:
# Confusion matrix
conf_matrix = metrics.confusion_matrix(y_true=pred_val["actual"].replace({genre_1: 1, genre_2: 0}), y_pred=pred_val["predicted"].replace({genre_1: 1, genre_2: 0}))
conf_matrix

array([[170,  30],
       [ 20, 180]])

In [8]:
# Metrics
accuracy = metrics.accuracy_score(y_true=pred_val["actual"], y_pred=pred_val["predicted"])
error = 1 - accuracy
TPR = conf_matrix[1,1] / (conf_matrix[1,1] + conf_matrix[1,0])
FPR = conf_matrix[0,1] / (conf_matrix[0,1] + conf_matrix[0,0])
TNR = conf_matrix[0,0] / (conf_matrix[0,0] + conf_matrix[0,1])
FNR = conf_matrix[1,0] / (conf_matrix[1,0] + conf_matrix[1,1])
print(f"Accuracy: {accuracy}\nError: {error}\nTPR: {TPR}\nFPR: {FPR}\nTNR: {TNR}\nFNR: {FNR}")

Accuracy: 0.875
Error: 0.125
TPR: 0.9
FPR: 0.15
TNR: 0.85
FNR: 0.1


In [9]:
# Predicted probability densities 
px.histogram(pred_val, x="prob", color="actual", opacity=0.5, barmode="overlay", title="Predicted Probability Densities")

In [10]:
# ROC Curve

lr_fpr, lr_tpr, lr_thresholds = metrics.roc_curve(y_true=pred_val["actual"], y_score=pred_val["prob"])


In [11]:
roc_lr = pd.DataFrame({"False Positive Rate": lr_fpr, "True Positive Rate": lr_tpr, "Model": "Logistic Regression"}, index=lr_thresholds)
roc_df = pd.concat([roc_lr]) # This concatenation is necessary for the plot to work
px.line(roc_df, y="True Positive Rate", x="False Positive Rate", color="Model", title="LR ROC Curve")

In [12]:
# Compute AUC
lr_auc = metrics.roc_auc_score(y_true=pred_val["actual"].replace({genre_1: 1, genre_2: 0}), y_score=pred_val["prob"])
print("Logistic Regression AUC:", lr_auc.round(3))

# The AUC measures the area under the ROC curve. The closer the AUC is to 1, the better the model is at distinguishing between the two classes.
# AUC is threshold independent - it summarizes performance over all possible classification thresholds
# The AUC is also robust to class imbalance, as it considers the trade-off between sensitivity and specificity (this makes it a more informative metric when class distributions are skewed)
# If a model's AUC is 0 <= X <= 1, it means that it can distinguish between positive and negative cases with a probability of X

Logistic Regression AUC: 0.946


In [13]:
# # Side note: forward selection code 
# from sklearn.feature_selection import SequentialFeatureSelector

# # Only look at a subset of the data to speed up the computation
# logistic_data_train_sub = logistic_data_train.iloc[:1000]

# selector = SequentialFeatureSelector(
#     lr_all,
#     n_features_to_select=4,
#     direction='forward',
#     scoring='accuracy',
#     cv = 5
# )

# selector.fit(X=logistic_data_train_sub.drop(columns="track_genre"), y=logistic_data_train_sub["track_genre"])
# selector.get_feature_names_out()

array(['danceability', 'acousticness', 'instrumentalness',
       'time_signature'], dtype=object)

In [14]:
# Using CV to evaluate model performance 
from sklearn.model_selection import cross_val_score
# predictor_variables = selector.get_feature_names_out()
predictor_variables = ['danceability', 'acousticness', 'instrumentalness','time_signature']

cross_val_score(lr_all, logistic_data_train[predictor_variables], logistic_data_train["track_genre"], cv=5, scoring="roc_auc")
# Accuracy for each fold:

array([0.99554688, 0.99697327, 0.99347484, 0.99544025, 0.99752358])

We chose the threshold for positive predictions by looking at the proportion of positive class labels in the training data. During the initial set up of the model, we chose the two genres that had the same proportion in the original data. Furthermore, when splitting the data into training and validation sets we used the stratify parameter to ensure that the proportion of the two genres was the same in both the training and validation sets. Thus our threshold for positive predictions is 0.5.